In [ ]:
"""
Decorators that accept parameters are parametrized decorators.
When parsing a decorator Python takes the decorated function
and passes it as the first argument to the decorator function.
So how do you make a decorator accept other arguments?
You must make a decorator factory function that accepts some
arguments and return a decorator which in turn will be applied
to the function to be decorated.
"""

In [2]:
registry = []


def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func


@register
def f1():
    print('running f1()')
    
print('running main()')
print('registry ->', registry)
f1()

running register(<function f1 at 0x7f6298681620>)
running main()
registry -> [<function f1 at 0x7f6298681620>]
running f1()


In [ ]:
# Will parametrize it accepting optional active parameter
# Which if set to false will skip registering the decorated functions

In [5]:
registry = set()


def register(active=True):
    def decorate(func):
        print(f'Running register(active={active}) -> decorate({func})')
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func   # this is decorator so must return a function
    return decorate  # a decorator factory must return a decorator

In [6]:
@register(active=False)
def f1():
    print('running f1()')


@register()
def f2():
    print('running f2()')


def f3():
    pass

Running register(active=False) -> decorate(<function f1 at 0x7f629860fbf8>)
Running register(active=True) -> decorate(<function f2 at 0x7f629860fb70>)


In [7]:
registry

{<function __main__.f2>}

In [8]:
f2()

running f2()


In [9]:
register()(f3)

Running register(active=True) -> decorate(<function f3 at 0x7f6298681620>)


<function __main__.f3>

In [10]:
registry

{<function __main__.f3>, <function __main__.f2>}

In [11]:
register(active=False)(f2)

Running register(active=False) -> decorate(<function f2 at 0x7f629860fb70>)


<function __main__.f2>

In [13]:
registry

{<function __main__.f3>}

In [22]:
"""This was a fairly simple example. As normally parametrized
decorators require yet another level of nesting. Will now 
change the clock decorator so user can pass in format string
to control the output."""
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'


def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args, **_kwargs):
            t0 = time.time()
            _result = func(*_args, **_kwargs)
            elapsed = time.time() - t0
            name = func.__name__
            result = _result
            args = ', '.join(repr(arg) for arg in _args)
            kwargs = ', '.join(f'{name}={value!r}' 
                               for name, value in _kwargs.items())
            print(fmt.format(**locals()))  # Wow we use locals dict!
            return _result
        return clocked
    return decorate


@clock()
def snooze(seconds):
    time.sleep(seconds)

In [23]:
for i in range(3):
    snooze(.123)

[0.12329626s] snooze(0.123) -> None
[0.12371731s] snooze(0.123) -> None


[0.12317824s] snooze(0.123) -> None


In [24]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(0.123)

snooze: 0.12317609786987305s
snooze: 0.12317633628845215s


snooze: 0.1232140064239502s


In [25]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(0.123)

snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s


snooze(0.123) dt=0.123s


In [34]:
# Non-trivial decorators are better implemented as classes
# The clock above would become


class clock:
    def __init__(self, fmt=DEFAULT_FMT):
        self.fmt = fmt
    
    def __call__(self, func):
        def clocked(*_args, **_kwargs):
            t0 = time.time()
            _result = func(*_args, **_kwargs)
            elapsed = time.time() - t0
            name = func.__name__
            result = _result
            args = ', '.join(repr(arg) for arg in _args)
            kwargs = ', '.join(f'{name}={value!r}' 
                               for name, value in _kwargs.items())
            print(self.fmt.format(**locals()))  # Wow we use locals dict!
            return _result
        return clocked

In [35]:
@clock('{name}____time_elapsed____:{elapsed:0.2f}')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(0.123)

snooze____time_elapsed____:0.12
snooze____time_elapsed____:0.12


snooze____time_elapsed____:0.12


In [36]:
# Funnily this is a wrong way to implement decorator factory
# using classes see Graham Dumpleton blog on decorators

In [ ]:
"""
To sum up to unerstand how decorators work you must be able to:
- Distinguish between import time and run time
- understand variable scoping, closures and nonlocal declaration
"""